<a href="https://colab.research.google.com/github/GavinHacker/recsys_model/blob/master/3_recbycontent_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install pymysql

    100% |████████████████████████████████| 51kB 4.5MB/s 


In [0]:
!pip install git+https://github.com/coreylynch/pyFM

  Cloning https://github.com/coreylynch/pyFM to /tmp/pip-req-build-ixrclqal
  Running setup.py bdist_wheel for pyfm ... - \ | / - \ done
  Stored in directory: /tmp/pip-ephem-wheel-cache-g1vyicl5/wheels/3b/d9/ef/1b148c527d39344632833679e79b3db1798a40b0f64f917b13
Successfully built pyfm


In [0]:
import pandas as pd
import pymysql
import pymysql.cursors
from functools import reduce
import numpy as np
import pandas as pd
import uuid
import datetime
from pyfm import pylibfm
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics.pairwise import pairwise_distances
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)

In [0]:
def get_connection():
    return pymysql.connect(host='rm-2zeqqm6994abi7b6dqo.mysql.rds.aliyuncs.com',
                               user='noone',
                               password='Huawei12#$',
                               db='recsys',
                               port=3306,
                               charset ='utf8',
                               use_unicode=True)

In [0]:
df = pd.read_sql_query("select * from movie", get_connection())

In [0]:
#df_exp = df.head(1)

## 对type, actors, director, trait四个类型数据分别构建一个频度统计字典，用于之后的one-hot编码

In [0]:
def get_dim_dict(df, dim_name):
  type_list = list(map(lambda x:x.split('|') ,df[dim_name]))
  type_list = [x for l in type_list for x in l]
  def reduce_func(x, y):
    for i in x:
      if i[0] == y[0][0]:
        x.remove(i)
        x.append(((i[0],i[1] + 1)))
        return x
    x.append(y[0])
    return x
  l = filter(lambda x:x != None, map(lambda x:[(x, 1)], type_list))
  type_zip = reduce(reduce_func, list(l))
  #type_list = sorted(list(set(type_list)))
  #type_zip = zip(list(range(len(type_list))), type_list)
  #print(len(type_zip))
  #print(type_zip)
  type_dict = {}
  for i in type_zip:
    type_dict[i[0]] = i[1]
  return type_dict

In [0]:
type_dict = get_dim_dict(df, 'type')
actors_dict = get_dim_dict(df, 'actors')
director_dict = get_dim_dict(df, 'director')
trait_dict = get_dim_dict(df, 'trait')

In [0]:
#director_dict['吉姆·弗尔']

In [0]:
#actors_dict['马克·莱昂纳蒂']

## 删除无用或冗余维度

In [0]:
df_ = df.drop(['ADD_TIME', 'enable', 'rat', 'id', 'name'], axis=1)

In [0]:
#df_.head()

## 将电影数据转换为字典列表，由于演员和导演均过万维，实际计算时过于稀疏，当演员或导演只出现一次时，标记为冷门演员或导演

In [0]:
movie_dict_list = []
for i in df_.index:
  movie_dict = {}
  #type
  for s_type in df_.iloc[i]['type'].split('|'):
    movie_dict[s_type] = 1
  #actors
  for s_actor in df_.iloc[i]['actors'].split('|'):
    if actors_dict[s_actor] < 2:
      movie_dict['other_actor'] = 1
    else:
      movie_dict[s_actor] = 1
  #regios
  movie_dict[df_.iloc[i]['region']] = 1
  #director
  for s_director in df_.iloc[i]['director'].split('|'):
    if director_dict[s_director] < 2:
      movie_dict['other_director'] = 1
    else:
      movie_dict[s_director] = 1
  #trait
  for s_trait in df_.iloc[i]['trait'].split('|'):
    movie_dict[s_trait] = 1
  movie_dict_list.append(movie_dict)

## 使用DictVectorizer进行向量化，做One-hot编码

In [0]:
v = DictVectorizer()
X = v.fit_transform(movie_dict_list)

In [0]:
X.shape

(6630, 5858)

In [0]:
movie_dict_list[0:1]

[{'other_actor': 1,
  '冒险': 1,
  '动作': 1,
  '奇幻': 1,
  '妮可·基德曼': 1,
  '威廉·达福': 1,
  '娜塔莉亚姗福瑞': 1,
  '帕特里克·威尔森': 1,
  '朱莉·安德鲁斯': 1,
  '杜夫·龙格尔': 1,
  '杰曼·翰苏': 1,
  '杰森·莫玛': 1,
  '格拉汉姆·麦克泰维什': 1,
  '温子仁': 1,
  '特穆拉·莫里森': 1,
  '约翰·瑞斯-戴维斯': 1,
  '美国': 1,
  '艾梅柏·希尔德': 1,
  '魔幻': 1}]

In [0]:
#pd.DataFrame(data=X.toarray(), columns=v.feature_names_).T.head(5)

## 计算cosine相似度

In [0]:
item_similarity = pairwise_distances(X, metric='cosine')

In [0]:
#item_similarity.shape

## 测试代码，查看相似度的准确性

In [0]:
compare_index = 3
index = 0
_max_index = 0
_max = 1
for i in item_similarity[compare_index]:
  if i < _max and i != 0:
    _max = i
    _max_index = index
  index = index + 1
print(_max_index, _max)

1707 0.4777670321329065


In [0]:
index_of_sim = _max_index

In [0]:
df.iloc[index_of_sim]
movie_dict_list[index_of_sim]
df_106 = pd.DataFrame(data=X.todense()[index_of_sim], columns=v.feature_names_)
df_0 = pd.DataFrame(data=X.todense()[compare_index], columns=v.feature_names_)
df_diff = pd.concat([df_0, df_106], axis=0, ignore_index=True)
#df_diff.reindex(['0','1'])
df_diff = df_diff.T
print(df_diff[(df_diff[0] != 0) | (df_diff[1] != 0)])

               0    1
Lady Gaga    1.0  0.0
other_actor  1.0  1.0
俄罗斯          1.0  1.0
剧情           0.0  1.0
动作           1.0  1.0
喜剧           1.0  0.0
奥列格·扬科夫斯基    0.0  1.0
女性           0.0  1.0
安东尼·威勒       0.0  1.0
悬疑           0.0  1.0
惊悚           1.0  1.0
搞笑           1.0  1.0
犯罪           1.0  1.0
米歇尔·罗德里格兹    1.0  0.0
索菲娅·维加拉      1.0  0.0
罗伯特·罗德里格兹    1.0  0.0
艾梅柏·希尔德      1.0  0.0


In [0]:
#df.iloc[index_of_sim]
#movie_dict_list[0]

In [0]:
#df.iloc[compare_index]

In [0]:
#item_similarity.shape

In [0]:
# li_dict = {1:0.1, 2:0.2, 5:0.4, 8:0.3}
# min(li_dict.values())
# dict(filter(lambda x:x[1] != 0.4 ,li_dict.items()))

In [0]:
df_sim = pd.DataFrame(data=item_similarity)

In [0]:
item_similarity.shape[0]

6630

## 最相似的前200个电影

In [0]:
df_sim.nsmallest(200, 6600)[6600].to_dict().items()

dict_items([(6600, 0.0), (2463, 0.36099034957730636), (405, 0.37720084467078163), (2796, 0.37720084467078163), (4625, 0.37720084467078163), (252, 0.39753592392329073), (3218, 0.40000000000000013), (988, 0.40371520600005606), (3605, 0.40371520600005606), (1706, 0.41445995623088017), (2691, 0.41905249806888767), (3364, 0.41905249806888767), (5731, 0.41905249806888767), (6087, 0.41905249806888767), (2540, 0.42710810076845374), (4116, 0.4284523933505918), (3661, 0.4363981380233656), (782, 0.4479475525261166), (2098, 0.4479475525261166), (3456, 0.4479475525261166), (5634, 0.4479475525261166), (3784, 0.45505073908693394), (4415, 0.45505073908693394), (654, 0.4666666666666668), (2100, 0.4666666666666668), (3660, 0.4666666666666668), (6255, 0.4666666666666668), (835, 0.47825080525004904), (1469, 0.47825080525004904), (2742, 0.47825080525004904), (2789, 0.47825080525004904), (2842, 0.47825080525004904), (3151, 0.47825080525004904), (3259, 0.47825080525004904), (3503, 0.47825080525004904), (4624

In [0]:
#df_sim.head(1)

## 插入一条基于内容推荐的电影关系数据，即，将基于相似度计算得出的电影插入数据库中的ibmovie表，此函数中主要参数分别代表：
原电影ID，与该电影类似的推荐电影ID，与该电影类似的推荐电影评分，相似度等

## ibmovie表：用于对外提供一个电影的相似电影集合

In [0]:
def insert_one_ibmovie(id_,
                      movieid,
                      recmovieid,
                      recmovierat,
                      simrat,
                      time,
                      enable, connection):
  sql = 'insert into ibmovie (id, movieid, recmovieid, recmovierat, simrat, time, enable) values (\'%s\',\'%s\',\'%s\',\'%s\',\'%s\',\'%s\',\'%s\')' % (id_, movieid, recmovieid, recmovierat, simrat, time, enable)
  #print(sql)
  try:
    with connection.cursor() as cursor:
        cout=cursor.execute(sql)
  except Exception as e:
    print('exception:'+str(e))
    try:
      connection.close()
    except:
      pass

## Test code

In [0]:
# _conn = get_connection()
# insert_one_ibmovie(uuid.uuid4(), '1', '2', '6', '0.1', datetime.datetime.now(), '1', _conn)
# _conn.commit()
# _conn.close()

In [0]:
# df_sim.nsmallest(10, 0)[0]

## 辅助函数，统计在ibmovie表中一个电影的相似电影数量

In [0]:
def get_recmovie_cnt_by_movieid(movieid, connection):
  sql = 'select count(*) from ibmovie group by movieid having movieid=\'%s\'' % movieid
  try:  
    with connection.cursor() as cursor:
      cout=cursor.execute(sql)
      if cout == 0:
        return 0
      return cursor.fetchone()[0]  
  except Exception as e:
    print('exception@get_recmovie_by_movieid:'+str(e))
    try:
      connection.cursor().close()
      connection.close()
      print('closed')
    except Exception as e1:
      print('exception1@get_recmovie_by_movieid:' + str(e1))
    connection = get_connection()

In [0]:
cnt = get_recmovie_cnt_by_movieid('10746430_', get_connection())
print(cnt)

In [0]:
df.iloc[2850]

In [0]:
start_index = 2849
start_index = 2961

## 离线处理ibmovie表，当有新的电影出现时，增量处理逻辑如下（这部分逻辑在recsys_core中实现）：

* 1.新电影的相似电影集合，即生成200个与新电影的相似电影

* 2.增量处理原有电影与新电影的相似度，每一个原有电影与新电影计算相似度，如果相似度大于某个原有电影的相似电影集合中相似度最小的那个，则插入新电影作为某原有电影的推荐电影。

In [0]:
def process_offline_compute_by_cosdis(rec_per_num):
  rec_per_num = rec_per_num + 1
  connection = get_connection()
  for i in range(start_index, item_similarity.shape[0]):
    df_sim_p = df_sim.nsmallest(rec_per_num, i)
    df_sim_p = df_sim_p[i]
    movie_id = df.iloc[i]['id']
    recmovie_cnt = get_recmovie_cnt_by_movieid(movie_id, connection)
    if recmovie_cnt == 200:
      print('org...')
      continue
    print('new...')
    time_now = datetime.datetime.now()
    for rec_movie_item in df_sim_p.to_dict().items():
      if rec_movie_item[0] != i:
        rec_movie_index = rec_movie_item[0]
        rec_movie_sim = rec_movie_item[1]
        rec_movie_id = df.iloc[rec_movie_index]['id']
        rec_movie_rat = df.iloc[rec_movie_index]['rat']
        insert_one_ibmovie(id_=uuid.uuid4(), movieid=movie_id, recmovieid=rec_movie_id, recmovierat=rec_movie_rat, simrat=rec_movie_sim, time=time_now, enable='1', connection=connection)
    connection.commit()

In [0]:
process_offline_compute_by_cosdis(rec_per_num=200)

In [0]:
# df_sim_3_dict = df_sim_3[4].to_dict()
# for i in df_sim_3_dict.items():
#   print(i)